In [4]:
from multiprocessing import Pool
from glob import glob
from tqdm.notebook import tqdm
import unicodedata
import os
import sys
def i1():# helper function for fork printing
  sys.stdout.write('*')
  sys.stdout.flush()

###Transform Greek-Bert and Legal-Bert dataset 
just like in the paper 'Greek-Bert'. code from https://github.com/nlpaueb/greek-bert. Changed slightly and executed localy

In [2]:
# https://github.com/nlpaueb/greek-bert/blob/master/build_data_el.py
def process_file(filename):
  lines = []
  with open(filename, 'r') as f:
    for line in f:
      lines.append(line.encode("utf-8", "ignore").decode())

  with open(filename, 'w') as f:
    for line in lines:
      if line.endswith('\n'):
        line = line.strip('\n')
      f.write(line + '\n')

# filenames = glob('/mnt/D21005A6100592A1/ΕΚΠΑ/πτυχιακή/Bert-final/datasets/Greek-Bert/1st_pass/*/*')
# with Pool(processes=2) as pool:
#   a = list(tqdm(pool.imap(process_file, filenames), total=len(filenames), desc='Greek-Bert'))
filenames = glob('/mnt/D21005A6100592A1/ΕΚΠΑ/πτυχιακή/Bert-final/datasets/Legal-Bert/1st_pass/*/*/*.txt')
with Pool(processes=16) as pool:
  a = list(tqdm(pool.imap(process_file, filenames), total=len(filenames), desc='Greek-Bert'))

In [9]:
# https://github.com/nlpaueb/greek-bert/blob/master/normalize_data.py
def _is_punctuation(char):
  """Checks whether `chars` is a punctuation character."""
  cp = ord(char)
  # We treat all non-letter/number ASCII as punctuation.
  # Characters such as "^", "$", and "`" are not in the Unicode
  # Punctuation class but we treat them as punctuation anyways, for
  # consistency.
  if ((cp >= 33 and cp <= 47) or (cp >= 58 and cp <= 64) or
      (cp >= 91 and cp <= 96) or (cp >= 123 and cp <= 126)):
    return True
  cat = unicodedata.category(char)
  if cat.startswith("P"):
    return True
  return False

def _run_split_on_punc(text):
  """Splits punctuation on a piece of text."""
  chars = list(text)
  i = 0
  start_new_word = True
  output = []
  while i < len(chars):
    char = chars[i]
    if _is_punctuation(char):
      output.append([char])
      start_new_word = True
    else:
      if start_new_word:
        output.append([])
      start_new_word = False
      output[-1].append(char)
    i += 1

  return ["".join(x) for x in output]

def strip_accents_and_lowercase(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
                if unicodedata.category(c) != 'Mn').lower()

def normalize(filename):
  output_file = open(filename.replace('/home/stratos/Downloads/1st_pass/',
                                      '/mnt/D21005A6100592A1/ΕΚΠΑ/πτυχιακή/Bert-final/datasets/Legal-Bert/normalized/'), 'w', encoding='utf8')
  with open(filename, encoding='utf8') as f:
    for line in f:
      tokens = line.lower().split()
      splited_tokens = []
      for token in tokens:
        splited_tokens.extend(_run_split_on_punc(token))
      line = ' '.join(splited_tokens)
      line = strip_accents_and_lowercase(line)
      if line.endswith('\n'):
        output_file.write(line)
      else:
        output_file.write(line+'\n')
  output_file.close()

# filenames = glob('/home/stratos/Downloads/1st_pass/*/*')
# with Pool(processes=4) as pool:
#   a = list(tqdm(pool.imap(normalize, filenames), total=len(filenames)))
filenames = glob('/home/stratos/Downloads/1st_pass/*/*/*.txt')
# for f in filenames:
#   f = f.replace('/home/stratos/Downloads/1st_pass/',
#             '/mnt/D21005A6100592A1/ΕΚΠΑ/πτυχιακή/Bert-final/datasets/Legal-Bert/normalized/')
#   os.makedirs(os.path.dirname(f), exist_ok=True)
with Pool(processes=8) as pool:
  a = list(tqdm(pool.imap(normalize, filenames), total=len(filenames)))

###Transform RAPTARCHIS dataset 
just like the previous one, just for dataframe.pkl

In [3]:
# https://github.com/nlpaueb/greek-bert/blob/master/build_data_el.py
import pandas as pd

def process_file(filename):
  lines = []
  df = pd.read_pickle(filename)
  for _, row in tqdm(df.iterrows(), total=len(df),
                      desc=f'loading {filename[-10:]}', leave=True):
    text = row['header'] + ' ' + row['articles']
    lines.append(text.encode("utf-8", "ignore").decode())

  new = []
  for line in lines:
    if line.endswith('\n'):
      line = line.strip('\n')
    new.append(line + '\n')
  df = pd.DataFrame(new, columns=['text'])
  output_file = filename.replace('RAPTARCHIS/','RAPTARCHIS/normalized/')
  df.to_pickle(output_file)


filenames = glob.glob('/mnt/D21005A6100592A1/ΕΚΠΑ/πτυχιακή/Bert-final/datasets/RAPTARCHIS/*.pkl')
with Pool(processes=3, initializer=i1) as pool:
  a = list(tqdm(pool.imap(process_file, filenames), total=len(filenames)))

***

In [4]:
def _is_punctuation(char):
  """Checks whether `chars` is a punctuation character."""
  cp = ord(char)
  # We treat all non-letter/number ASCII as punctuation.
  # Characters such as "^", "$", and "`" are not in the Unicode
  # Punctuation class but we treat them as punctuation anyways, for
  # consistency.
  if ((cp >= 33 and cp <= 47) or (cp >= 58 and cp <= 64) or
      (cp >= 91 and cp <= 96) or (cp >= 123 and cp <= 126)):
    return True
  cat = unicodedata.category(char)
  if cat.startswith("P"):
    return True
  return False

def _run_split_on_punc(text):
  """Splits punctuation on a piece of text."""
  chars = list(text)
  i = 0
  start_new_word = True
  output = []
  while i < len(chars):
    char = chars[i]
    if _is_punctuation(char):
      output.append([char])
      start_new_word = True
    else:
      if start_new_word:
        output.append([])
      start_new_word = False
      output[-1].append(char)
    i += 1

  return ["".join(x) for x in output]

def strip_accents_and_lowercase(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
                if unicodedata.category(c) != 'Mn').lower()

def normalize(filename):
  df = pd.read_pickle(filename)
  new = []
  for _, line in tqdm(df.iterrows(), total=len(df),
                      desc=f'loading {filename[-10:]}', leave=True):
    line = line['text']
    tokens = line.lower().split()
    splited_tokens = []
    for token in tokens:
      splited_tokens.extend(_run_split_on_punc(token))
    line = ' '.join(splited_tokens)
    line = strip_accents_and_lowercase(line)
    if line.endswith('\n'):
      new.append(line)
    else:
      new.append(line+'\n')
  df = pd.DataFrame(new, columns=['text'])
  df.to_pickle(filename)

filenames = glob.glob('/mnt/D21005A6100592A1/ΕΚΠΑ/πτυχιακή/Bert-final/datasets/RAPTARCHIS/normalized/*.pkl')
with Pool(processes=3, initializer=i1) as pool:
  a = list(tqdm(pool.imap(normalize, filenames), total=len(filenames),
                desc='files', leave=True))

***